# Spherical Projections

In [ ]:
#| default_exp spherical_projections

In [ ]:
#| hide
from nbdev.showdoc import *

In [ ]:
#| export
import matplotlib.pyplot as plt
import numpy as np

Realiza a conversão de uma point cloud 3d para o plano 2d através da técnica de projeção esférica.

* Args:
    * array_npy: array numpy contendo os pontos da imagem, shape precisa ser (N,4).
    * val: define a lista de pixel_values, responsável por dar um valor para cada ponto.
* Return:
    * points: Dicionário contendo os novos pontos já projetados no plano 2d.

In [ ]:
#| export
def spherical_projection_1(array_npy, h_res, v_res, v_fov, y_fudge, val):
    x_lidar = array_npy[:, 0]
    y_lidar = array_npy[:, 1]
    z_lidar = array_npy[:, 2]
    r_lidar = array_npy[:, 3] # Reflectance
    if array_npy.shape[-1] >= 5:
        label = array_npy[:, 4]
    
    # Distance relative to origin when looked from top
    d_lidar = np.sqrt(x_lidar ** 2 + y_lidar ** 2)

    v_fov_total = -v_fov[0] + v_fov[1]

    # Convert to Radians
    v_res_rad = v_res * (np.pi/180)
    h_res_rad = h_res * (np.pi/180)

    # PROJECT INTO IMAGE COORDINATES
    x_img = np.arctan2(-y_lidar, x_lidar)/ h_res_rad
    y_img = np.arctan2(z_lidar, d_lidar)/ v_res_rad
    
    # SHIFT COORDINATES TO MAKE 0,0 THE MINIMUM
    x_min = -360.0 / h_res / 2  # Theoretical min x value based on sensor specs
    x_img -= x_min              # Shift
    x_max = 360.0 / h_res       # Theoretical max x value after shifting

    y_min = v_fov[0] / v_res    # theoretical min y value based on sensor specs
    y_img -= y_min              # Shift
    y_max = v_fov_total / v_res # Theoretical max x value after shifting

    y_max += y_fudge            # Fudge factor if the calculations based on
                                # spec sheet do not match the range of
                                # angles collected by in the data.

    if val == "reflectance":
        pixel_values = r_lidar
    elif val == "height":
        pixel_values = z_lidar
    elif val == "label":
        pixel_values = label
    else:
        pixel_values = -d_lidar

    points = {
        'x_img': x_img,
        'y_img': y_img,
        'x_max': x_max,
        'y_max': y_max,
        'val': val,
        'pixel_values': pixel_values
    }
    
    return points

In [ ]:
#| export
'''
- Para a projeção esférica são lidos as coordenadas x,y,z de cada ponto
- R é a distância do laser até a origem (LiDAR)
- desvio_eixoY é o valor no plano xy e o ângulo com range de [FOV_UP, FOV_DOWN] para max/min
- desvio_eixoX é o valor no plano xz com range de [-pi,pi] por causa da função arctan2
- É necessário normalizar os desvios para pegar os pontos na distorção para 2D
- Como o modelo do LiDAR utilizado é o Velodyne HDL 64-E, a imagem deve ser 64x1024
- Retorna o ponto (normal_Y,normal_X, intensidade) da projeção
'''

def spherical_projection_2(point_cloud):
    x_coord = point_cloud[:, 0]
    y_coord = point_cloud[:, 1]
    z_coord = point_cloud[:, 2]

    R = np.sqrt(x_coord**2 + y_coord**2 + z_coord**2)

    desvio_eixoY = np.arcsin(z_coord/R)
    desvio_eixoX = np.arctan2(y_coord, x_coord)

    FOV_UP = np.max(desvio_eixoY)
    FOV_DOWN = np.min(desvio_eixoY)
    FOV = FOV_UP + abs(FOV_DOWN)

    normal_Y = 1.0 - ((desvio_eixoY + abs(FOV_DOWN))/FOV)
    normal_X = 0.5 * ((desvio_eixoX/np.pi) + 1)

    return normal_Y*64, normal_X*1024, point_cloud[:, 3], point_cloud[:, 4]

In [ ]:
#| export
'''
- A imagem é formada no plano zy
- É necessário truncar os valores para servirem como índice na matriz
- Retorna a intensidade de cada ponto para formar a imagem
'''
def create_ImageMatrix(normal_Y, normal_X, intensity, label):
    truncated_Y = np.minimum(63, np.round(normal_Y))
    index_Y = np.maximum(0, truncated_Y).astype(int)
    
    truncated_X = np.minimum(1023, np.round(normal_X))
    index_X = np.maximum(0, truncated_X).astype(int)
    
    image_matrix_no_label = np.zeros((64, 1024))
    image_matrix_with_label = np.zeros((64, 1024))
    
    for x, y, i in zip(index_X, index_Y, intensity):
        image_matrix_no_label[y, x] = i
    
    for x, y, i in zip(index_X, index_Y, label):
        image_matrix_with_label[y, x] = i
        
    return image_matrix_no_label, image_matrix_with_label

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()